In [1]:
import schedule
import time
from gtts import gTTS
import os
from playsound import playsound
playsound('C:/Users/joeca/Desktop/Finance/FOREX/Hercules_Prototype_Mark_II/MP3s/startup.mp3')

In [2]:
import winsound
import numpy as np
import pandas as pd
import math
from scipy import stats
import statsmodels.api as sm
import matplotlib.pyplot as plt
from sympy import *
init_printing()
import xlrd
from datetime import datetime as dt

In [3]:
from multiprocessing import Process
import json
from scipy.signal import argrelextrema
import pytz
import itertools
from itertools import combinations
from itertools import product
from itertools import groupby
from astropy.table import QTable, Table, Column
from astropy import units as u
from IPython.display import HTML

In [4]:
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import MetaTrader5 as mt5

In [5]:
if not mt5.initialize(path="C:/JoeC/MetaTrader 5/terminal64.exe",login=1045610, server="TradersWay-Demo",password="r5AH8znU"):
    print("initialize() failed, error code =",mt5.last_error())
    quit()

In [6]:
def brutus2(h,i,j,k,l,m,threshold,threshold2,RATE,RATE2):
    
    
    riskusd = 5.00

    spy = "YES"
    
    symbol_info = mt5.symbol_info(h)
    point = mt5.symbol_info(h).point
    base, quote = h[:3], h[3:]
    
    if quote == "JPY":
        stpips = abs(l - j)*100
    else:
        stpips = abs(l - j)*10000
    
    shrink = 0.90

    if i == 'LONG':
        tpdist = j + k
        k = j + tpdist*shrink
    elif i == 'SHORT':
        tpdist = j - k
        k = j - tpdist*shrink

    if i == 'LONG':
        tpdist = k - j
        sldist = j - l
        if sldist > tpdist:
            l = j - tpdist
    elif i == 'SHORT':
        tpdist = j - k
        sldist = l - j
        if sldist > tpdist:
            l = j + tpdist

    if i == "LONG":
        price = mt5.symbol_info_tick(h).bid
        if base == "USD":
            exchange = 1.00
        if quote == "USD":
            exchange = 1.00
        elif quote == "AUD":
            exchange = mt5.symbol_info("AUDUSD").bid
            exchange = 1/exchange
        elif quote == "CAD":
            exchange = mt5.symbol_info("USDCAD").bid
        elif quote == "CHF":
            exchange = mt5.symbol_info("USDCHF").bid
        elif quote == "EUR":
            exchange = mt5.symbol_info("EURUSD").bid
            exchange = 1/exchange
        elif quote == "GBP":
            exchange = mt5.symbol_info("GBPUSD").bid
            exchange = 1/exchange
        elif quote == "JPY":
            exchange = mt5.symbol_info("USDJPY").bid
        elif quote == "NZD":
            exchange = mt5.symbol_info("NZDUSD").bid
            exchange = 1/exchange
        lot = round(((riskusd*exchange)/stpips)/10,2)
        request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": h,
            "volume": lot,
            "type": mt5.ORDER_TYPE_BUY,
            "price": price,
            "sl": l,
            "tp": k,
            "deviation": 20,
            "magic": 234000,
            "comment": "BRUTUS Mark V",
            "type_time": mt5.ORDER_TIME_GTC,
            "type_filling": mt5.ORDER_FILLING_IOC,
        }
        extraj = j + m
        selected=mt5.symbol_select(h,True)
        if not selected:
            print("Failed to select pair")
            mt5.shutdown()
            quit()
            spy = "NO"

        lasttickb=mt5.symbol_info_tick(h).bid
        if lasttickb > extraj:
            RATE2 = RATE2 + 1
        else:
            RATE2 = 0
        if lasttickb > j:
            RATE = RATE + 1
        else:
            RATE = 0
        if RATE >= threshold:
            result = mt5.order_send(request)
            spy = "NO"
        if RATE2 >= threshold2:
            result = mt5.order_send(request)
            spy = "NO"

    elif i == "SHORT":
        price = mt5.symbol_info_tick(h).ask
        if quote == "USD":
            exchange = 1.00
        elif base == "USD":
            exchange = 1.00
        elif base == "AUD":
            exchange = mt5.symbol_info("AUDUSD").bid
            exchange = 1/exchange
        elif base == "CAD":
            exchange = mt5.symbol_info("USDCAD").bid
        elif base == "CHF":
            exchange = mt5.symbol_info("USDCHF").bid
        elif base == "EUR":
            exchange = mt5.symbol_info("EURUSD").bid
            exchange = 1/exchange
        elif base == "GBP":
            exchange = mt5.symbol_info("GBPUSD").bid
            exchange = 1/exchange
        elif base == "JPY":
            exchange = mt5.symbol_info("USDJPY").bid
        elif base == "NZD":
            exchange = mt5.symbol_info("NZDUSD").bid
            exchange = 1/exchange
        lot = round(((riskusd*exchange)/stpips)/10,2)
        request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": h,
            "volume": lot,
            "type": mt5.ORDER_TYPE_SELL,
            "price": price,
            "sl": l,
            "tp": k,
            "deviation": 20,
            "magic": 234000,
            "comment": "BRUTUS Mark V",
            "type_time": mt5.ORDER_TIME_GTC,
            "type_filling": mt5.ORDER_FILLING_IOC,
        }
        extraj = j - m
        selected=mt5.symbol_select(h,True)

        if not selected:
            print("Failed to select pair")
            mt5.shutdown()
            quit()
            spy = "NO"

        lasttickb=mt5.symbol_info_tick(h).bid
        if lasttickb < extraj:
            RATE2 = RATE2 + 1
        else:
            RATE2 = 0
        if lasttickb < j:
            RATE = RATE + 1
        else:
            RATE = 0
        if RATE >= threshold:
            result = mt5.order_send(request)
            spy = "NO"
        if RATE2 >= threshold2:
            result = mt5.order_send(request)
            spy = "NO"
            
    return RATE, RATE2, spy

In [7]:
## WATCHLIST = ['AUD/CAD', 'AUD/CHF', 'AUD/JPY', 'AUD/NZD', 'AUD/USD', 'CAD/CHF', 'CAD/JPY', 'CHF/JPY', 'EUR/AUD', 'EUR/CAD', 'EUR/CHF', 'EUR/GBP', 'EUR/JPY', 'EUR/NZD', 'EUR/USD', 'GBP/AUD', 'GBP/CAD', 'GBP/CHF', 'GBP/JPY', 'GBP/NZD', 'GBP/USD', 'NZD/CAD', 'NZD/CHF', 'NZD/JPY', 'NZD/USD', 'USD/CAD', 'USD/CHF', 'USD/JPY', 'XAG/USD', 'XAU/USD', 'BCH/USD', 'BTC/USD', 'EOS/USD', 'ETH/USD', 'LTC/USD', 'XLM/USD', 'XRP/USD']

## FOREX = ## WATCHLIST = ['AUDCAD','AUDCHF','AUDJPY','AUDNZD','AUDUSD','CADCHF','CADJPY','CHFJPY','EURAUD','EURCAD','EURCHF','EURGBP','EURJPY','EURNZD','EURUSD','GBPAUD','GBPCAD','GBPCHF','GBPJPY','GBPUSD','NZDCAD','NZDCHF','NZDJPY','NZDUSD','USDCAD','USDCHF','USDJPY']

def brutus():

    start = dt.now()
    purp = []
    pair = ['AUDCAD','AUDCHF','AUDJPY','AUDNZD','AUDUSD','CADCHF','CADJPY','CHFJPY','EURAUD','EURCAD','EURCHF','EURGBP','EURJPY','EURNZD','EURUSD','GBPAUD','GBPCAD','GBPCHF','GBPJPY','GBPUSD','NZDCAD','NZDCHF','NZDJPY','NZDUSD','USDCAD','USDCHF','USDJPY']
    l = 600
    lin = np.linspace(1,l,num=l)
    
    for x in pair:

        Srates = mt5.copy_rates_from_pos(x, mt5.TIMEFRAME_H8, 0, 600)
        S = Srates
        SCM = np.average(abs(S['close'] - S['open']))
        lx = S['low']
        lelx = lx[argrelextrema(lx, np.less)[0]]
        ly = S['high']
        lely = ly[argrelextrema(ly, np.greater)[0]]
        LVL = np.concatenate((lelx,lely))
        
        B = list(combinations(LVL, 2))
        Ba = np.array([x for x, y in B])
        B = np.array([abs(x - y) for x, y in B])
        Bb = np.array([np.where(x <= SCM, 1, 0) for x in B])
        T = list(zip(Ba,Bb))
        table = pd.DataFrame(T, columns=['a','b']).groupby('a').sum()
        table['max'] = table.b[(table.b.shift(1) < table.b) & (table.b.shift(-1) < table.b)]
        table = table.sort_values(['a'], ascending=False)
        table = table.fillna(0)
        table = table[table['max'] > np.average(table['b'])]
        table = table.assign(level=table.index)
        levels = list(table['level'])

        Drates = mt5.copy_rates_from_pos(x, mt5.TIMEFRAME_M5, 0, 600)
        D = pd.DataFrame(Drates)
        D['time']=pd.to_datetime(D['time'], unit='s')
        D = D.drop(columns=['tick_volume', 'spread', 'real_volume'])
        D = D.set_index('time')
        D = D.tz_localize('Etc/GMT-7')
        D = D.tz_convert('UTC')
        D = D.assign(MOVE=(D['close'] - D['open']))
        D = D.assign(STICK=(D['high'] - D['low']))
        D = D.assign(PSTICK=abs(D['MOVE']/D['STICK']))
        D['DIRECT'] = np.where(D.MOVE > 0, 'Bullish', 'Bearish')
        CM = np.average(abs(D['MOVE']))

        ANIMAL = []
        i = 0
        for t in lin: 
            if D.iloc[i,6] <= 1.1:
                if D.iloc[i,6] < 0.85:
                    if D.iloc[i,6] < 0.65:
                        if D.iloc[i,6] < 0.50:
                            if D.iloc[i,6] <= 0.30:
                                if D.iloc[i,6] <= 0.15:
                                    animal = 'Doji'
                                else:
                                    animal = 'Frog'
                            else:
                                animal = 'Cat'
                        else:
                            animal = 'Dog'
                    else:
                        animal = 'Horse'
                else:
                    animal = 'Shark'
            ANIMAL.append(animal)
            i = i + 1
        D['ANIMAL'] = ANIMAL

        GAUGE = []
        i = 0
        cndmid = (D.iloc[i,0]+D.iloc[i,3])/2
        stickmid = (D.iloc[i,1]+D.iloc[i,2])/2
        for t in lin: 
            if D.iloc[i,5] == 0:
                midpoint = D.iloc[i,0]
            else:
                midpoint = (cndmid - stickmid)/D.iloc[i,5]
            if midpoint >= 0.075:
                if midpoint >= 0.175:
                    if midpoint >= 0.25:
                        gauge = 'Top Saturated'
                    else:
                        gauge = 'Top Stacked'
                else:
                    gauge = 'Top Semi-Stacked'
            elif midpoint <= -0.075:
                if midpoint <= -0.175:
                    if midpoint <= -0.25:
                        gauge = 'Bottom Saturated'
                    else:
                        gauge = 'Bottom Stacked'
                else:
                    gauge = 'Bottom Semi-Stacked'
            else:
                gauge = 'Centered'
            GAUGE.append(gauge)
            i = i + 1
        D['GAUGE'] = GAUGE

        thh = 168
        tl = l - thh
        tlin = np.linspace(1,tl,num=tl)
        tlinb = np.linspace(1,thh,num=thh)
        TREND = []
        for t in tlinb:
            TREND.append(0)
        i = thh                                                                                       
        for t in tlin:
            ii = i - thh
            movetrend = np.sum(list(D.iloc[ii:i,4]))
            i = i + 1
            TREND.append(movetrend)
        D['TREND'] = TREND

        READING=[]
        i = 0
        for t in lin: 
            if D.iloc[i,10] < 0:
                if D.iloc[i,9] == 'Top Saturated':
                    if D.iloc[i,8] == 'Shark':
                        Read = 'MARIBOZU'
                    elif D.iloc[i,8] == 'Horse':
                        Read = 'TUBE'
                    elif D.iloc[i,8] == 'Dog':
                        Read = 'JACK'
                    elif D.iloc[i,8] == 'Cat':
                        Read = 'DIPPER'
                    elif D.iloc[i,8] == 'Frog':
                        Read = 'HAMMER'
                    elif D.iloc[i,8] == 'Doji':
                        Read = 'DRAGONFLY DOJI'
                    else:
                        Read = D.iloc[i,8]
                elif D.iloc[i,9] == 'Top Stacked':
                    if D.iloc[i,8] == 'Shark':
                        Read = 'MARIBOZU'
                    elif D.iloc[i,8] == 'Horse':
                        Read = 'TUBE'
                    elif D.iloc[i,8] == 'Dog':
                        Read = 'JACK'
                    elif D.iloc[i,8] == 'Cat':
                        Read = 'SKILLET'
                    elif D.iloc[i,8] == 'Frog':
                        Read = 'MALLET'
                    elif D.iloc[i,8] == 'Doji':
                        Read = 'DRAGONFLY DOJI'
                    else:
                        Read = D.iloc[i,8]
                elif D.iloc[i,9] == 'Top Semi-Stacked':
                    if D.iloc[i,8] == 'Shark':
                        Read = 'MARIBOZU'
                    elif D.iloc[i,8] == 'Horse':
                        Read = 'TUBE'
                    elif D.iloc[i,8] == 'Dog':
                        Read = 'GLIDE'
                    elif D.iloc[i,8] == 'Cat':
                        Read = 'GLIDE'
                    elif D.iloc[i,8] == 'Frog':
                        Read = 'PLUMP DOJI'
                    elif D.iloc[i,8] == 'Doji':
                        Read = 'LONG-LEG DOJI'
                    else:
                        Read = D.iloc[i,8]
                elif D.iloc[i,9] == 'Centered':
                    if D.iloc[i,8] == 'Shark':
                        Read = 'MARIBOZU'
                    elif D.iloc[i,8] == 'Horse':
                        Read = 'TUBE'
                    elif D.iloc[i,8] == 'Dog':
                        Read = 'GLIDE'
                    elif D.iloc[i,8] == 'Cat':
                        Read = 'GLIDE'
                    elif D.iloc[i,8] == 'Frog':
                        Read = 'PLUMP DOJI'
                    elif D.iloc[i,8] == 'Doji':
                        Read = 'LONG-LEG DOJI'
                    else:
                        Read = D.iloc[i,8]
                elif D.iloc[i,9] == 'Bottom Semi-Stacked':
                    if D.iloc[i,8] == 'Shark':
                        Read = 'MARIBOZU'
                    elif D.iloc[i,8] == 'Horse':
                        Read = 'TUBE'
                    elif D.iloc[i,8] == 'Dog':
                        Read = 'GLIDE'
                    elif D.iloc[i,8] == 'Cat':
                        Read = 'GLIDE'
                    elif D.iloc[i,8] == 'Frog':
                        Read = 'PLUMP DOJI'
                    else:
                        Read = D.iloc[i,8]
                elif D.iloc[i,9] == 'Bottom Stacked':
                    if D.iloc[i,8] == 'Shark':
                        Read = 'MARIBOZU'
                    elif D.iloc[i,8] == 'Horse':
                        Read = 'TUBE'
                    elif D.iloc[i,8] == 'Dog':
                        Read = 'BOOST'
                    elif D.iloc[i,8] == 'Cat':
                        Read = 'GLIDE'
                    elif D.iloc[i,8] == 'Frog':
                        Read = 'BRAKE'
                    elif D.iloc[i,8] == 'Doji':
                        Read = 'BRAKE'
                    else:
                        Read = D.iloc[i,8]
                elif D.iloc[i,9] == 'Bottom Saturated':
                    if D.iloc[i,8] == 'Shark':
                        Read = 'MARIBOZU'
                    elif D.iloc[i,8] == 'Horse':
                        Read = 'TUBE'
                    elif D.iloc[i,8] == 'Dog':
                        Read = 'BOOST'
                    elif D.iloc[i,8] == 'Cat':
                        Read = 'GLIDE'
                    elif D.iloc[i,8] == 'Frog':
                        Read = 'BRAKE'
                    elif D.iloc[i,8] == 'Doji':
                        Read = 'BRAKE'
                    else:
                        Read = D.iloc[i,8]
                else:
                    Read = D.iloc[i,8]
            elif D.iloc[i,10] > 0:
                if D.iloc[i,9] == 'Bottom Saturated':
                    if D.iloc[i,8] == 'Shark':
                        Read = 'MARIBOZU'
                    elif D.iloc[i,8] == 'Horse':
                        Read = 'TUBE'
                    elif D.iloc[i,8] == 'Dog':
                        Read = 'JACK'
                    elif D.iloc[i,8] == 'Cat':
                        Read = 'DIPPER'
                    elif D.iloc[i,8] == 'Frog':
                        Read = 'SHOOTING STAR'
                    elif D.iloc[i,8] == 'Doji':
                        Read = 'GRAVESTONE DOJI'
                    else:
                        Read = D.iloc[i,8]
                elif D.iloc[i,9] == 'Bottom Stacked':
                    if D.iloc[i,8] == 'Shark':
                        Read = 'MARIBOZU'
                    elif D.iloc[i,8] == 'Horse':
                        Read = 'TUBE'
                    elif D.iloc[i,8] == 'Dog':
                        Read = 'JACK'
                    elif D.iloc[i,8] == 'Cat':
                        Read = 'SKILLET'
                    elif D.iloc[i,8] == 'Frog':
                        Read = 'ASTEROID'
                    elif D.iloc[i,8] == 'Doji':
                        Read = 'GRAVESTONE DOJI'
                    else:
                        Read = D.iloc[i,8]
                elif D.iloc[i,9] == 'Bottom Semi-Stacked':
                    if D.iloc[i,8] == 'Shark':
                        Read = 'MARIBOZU'
                    elif D.iloc[i,8] == 'Horse':
                        Read = 'TUBE'
                    elif D.iloc[i,8] == 'Dog':
                        Read = 'GLIDE'
                    elif D.iloc[i,8] == 'Cat':
                        Read = 'GLIDE'
                    elif D.iloc[i,8] == 'Frog':
                        Read = 'PLUMP DOJI'
                    elif D.iloc[i,8] == 'Doji':
                        Read = 'LONG-LEG DOJI'
                    else:
                        Read = D.iloc[i,8]
                elif D.iloc[i,9] == 'Centered':
                    if D.iloc[i,8] == 'Shark':
                        Read = 'MARIBOZU'
                    elif D.iloc[i,8] == 'Horse':
                        Read = 'TUBE'
                    elif D.iloc[i,8] == 'Dog':
                        Read = 'GLIDE'
                    elif D.iloc[i,8] == 'Cat':
                        Read = 'GLIDE'
                    elif D.iloc[i,8] == 'Frog':
                        Read = 'PLUMP DOJI'
                    elif D.iloc[i,8] == 'Doji':
                        Read = 'LONG-LEG DOJI'
                    else:
                        Read = D.iloc[i,8]
                elif D.iloc[i,9] == 'Top Semi-Stacked':
                    if D.iloc[i,8] == 'Shark':
                        Read = 'MARIBOZU'
                    elif D.iloc[i,8] == 'Horse':
                        Read = 'TUBE'
                    elif D.iloc[i,8] == 'Dog':
                        Read = 'GLIDE'
                    elif D.iloc[i,8] == 'Cat':
                        Read = 'GLIDE'
                    elif D.iloc[i,8] == 'Frog':
                        Read = 'PLUMP DOJI'
                    elif D.iloc[i,8] == 'Doji':
                        Read = 'LONG-LEG DOJI'
                    else:
                        Read = D.iloc[i,8]
                elif D.iloc[i,9] == 'Top Stacked':
                    if D.iloc[i,8] == 'Shark':
                        Read = 'MARIBOZU'
                    elif D.iloc[i,8] == 'Horse':
                        Read = 'TUBE'
                    elif D.iloc[i,8] == 'Dog':
                        Read = 'BOOST'
                    elif D.iloc[i,8] == 'Cat':
                        Read = 'GLIDE'
                    elif D.iloc[i,8] == 'Frog':
                        Read = 'BRAKE'
                    elif D.iloc[i,8] == 'Doji':
                        Read = 'BRAKE'
                    else:
                        Read = D.iloc[i,8]
                elif D.iloc[i,9] == 'Top Saturated':
                    if D.iloc[i,8] == 'Shark':
                        Read = 'MARIBOZU'
                    elif D.iloc[i,8] == 'Horse':
                        Read = 'TUBE'
                    elif D.iloc[i,8] == 'Dog':
                        Read = 'BOOST'
                    elif D.iloc[i,8] == 'Cat':
                        Read = 'GLIDE'
                    elif D.iloc[i,8] == 'Frog':
                        Read = 'BRAKE'
                    elif D.iloc[i,8] == 'Doji':
                        Read = 'BRAKE'
                    else:
                        Read = D.iloc[i,8]
                else:
                    Read = D.iloc[i,8]
            else:
                Read = D.iloc[i,8]
            i = i + 1
            READING.append(Read)
        D['READING'] = READING

        READING2=[]
        READING2.append(0)
        jl = l - 1
        linjj = np.linspace(1,jl,num=jl)
        i = 1
        for t in linjj: 
            ii = i - 1
            if D.iloc[i,10] < 0:
                if D.iloc[ii,7] == 'Bearish':
                    if D.iloc[i,3] > D.iloc[ii,1]:
                        if D.iloc[ii,11] == 'BRAKE':
                            if D.iloc[i,7] == 'Bullish':
                                if D.iloc[i,11] == 'MARIBOZU':
                                    Read2 = 'BULLISH ENGULFING'
                                elif D.iloc[i,11] == 'TUBE':
                                    Read2 = 'BULLISH ENGULFING'
                                else:
                                    Read2 = 0
                            else:
                                Read2 = 0
                        elif D.iloc[ii,11] == 'GLIDE':
                            if D.iloc[i,7] == 'Bullish':
                                if D.iloc[i,11] == 'MARIBOZU':
                                    Read2 = 'BULLISH ENGULFING'
                                elif D.iloc[i,11] == 'TUBE':
                                    Read2 = 'BULLISH ENGULFING'
                                else:
                                    Read2 = 0
                            else:
                                Read2 = 0
                        elif D.iloc[ii,11] == 'PLUMP DOJI':
                            if D.iloc[i,7] == 'Bullish':
                                if D.iloc[i,11] == 'MARIBOZU':
                                    Read2 = 'BULLISH ENGULFING'
                                elif D.iloc[i,11] == 'TUBE':
                                    Read2 = 'BULLISH ENGULFING'
                                else:
                                    Read2 = 0
                            else:
                                Read2 = 0
                        elif D.iloc[ii,11] == 'LONG-LEG DOJI':
                            if D.iloc[i,7] == 'Bullish':
                                if D.iloc[i,11] == 'MARIBOZU':
                                    Read2 = 'BULLISH ENGULFING'
                                elif D.iloc[i,11] == 'TUBE':
                                    Read2 = 'BULLISH ENGULFING'
                                else:
                                    Read2 = 0
                            else:
                                Read2 = 0
                        else:
                            Read2 = 0
                    elif D.iloc[i,0] == D.iloc[ii,3]:
                        if D.iloc[i,2] == D.iloc[ii,2]:
                            if D.iloc[i,7] == 'Bullish':
                                if D.iloc[i,3] > D.iloc[ii,0]:
                                    Read2 = 'TWEEZERS BOTTOM'
                                else:
                                    Read2 = 0
                            else:
                                Read2 = 0
                        else:
                            Read2 = 0
                    elif D.iloc[i,3] > (D.iloc[ii,0]+D.iloc[ii,3])/2:
                        if D.iloc[i,0] < D.iloc[ii,3]:
                            if D.iloc[ii,7] == 'Bullish':
                                Read2 = 'PIERCING PATTERN'
                            else:
                                Read2 = 0
                        else:
                            Read2 = 0
                    else:
                        Read2 = 0
                else:
                    Read2 = 0
            elif D.iloc[i,10] > 0:
                if D.iloc[ii,7] == 'Bullish':
                    if D.iloc[i,3] < D.iloc[ii,2]:
                        if D.iloc[ii,11] == 'BRAKE':
                            if D.iloc[i,7] == 'Bearish':
                                if D.iloc[i,11] == 'MARIBOZU':
                                    Read2 = 'BEARISH ENGULFING'
                                elif D.iloc[i,11] == 'TUBE':
                                    Read2 = 'BEARISH ENGULFING'
                                else:
                                    Read2 = 0
                            else:
                                Read2 = 0
                        elif D.iloc[ii,11] == 'GLIDE':
                            if D.iloc[i,7] == 'Bearish':
                                if D.iloc[i,11] == 'MARIBOZU':
                                    Read2 = 'BEARISH ENGULFING'
                                elif D.iloc[i,11] == 'TUBE':
                                    Read2 = 'BEARISH ENGULFING'
                                else:
                                    Read2 = 0
                            else:
                                Read2 = 0
                        elif D.iloc[ii,11] == 'PLUMP DOJI':
                            if D.iloc[i,7] == 'Bearish':
                                if D.iloc[i,11] == 'MARIBOZU':
                                    Read2 = 'BEARISH ENGULFING'
                                elif D.iloc[i,11] == 'TUBE':
                                    Read2 = 'BEARISH ENGULFING'
                                else:
                                    Read2 = 0
                            else:
                                Read2 = 0
                        elif D.iloc[ii,11] == 'LONG-LEG DOJI':
                            if D.iloc[i,7] == 'Bearish':
                                if D.iloc[i,11] == 'MARIBOZU':
                                    Read2 = 'BEARISH ENGULFING'
                                elif D.iloc[i,11] == 'TUBE':
                                    Read2 = 'BEARISH ENGULFING'
                                else:
                                    Read2 = 0
                            else:
                                Read2 = 0
                        else:
                            Read2 = 0
                    else:
                        Read2 = 0
                elif D.iloc[i,3] == D.iloc[ii,0]:
                    if D.iloc[i,1] == D.iloc[ii,1]:
                        if D.iloc[i,7] == 'Bearish':
                            if D.iloc[i,3] < D.iloc[ii,0]:
                                Read2 = 'TWEEZERS TOP'
                            else:
                                Read2 = 0
                        else:
                            Read2 = 0
                    else:
                        Read2 = 0
                elif D.iloc[i,3] < (D.iloc[ii,0]+D.iloc[ii,3])/2:
                    if D.iloc[i,0] > D.iloc[ii,3]:
                        if D.iloc[ii,7] == 'Bearish':
                            Read2 = 'DARK CLOUD COVER'
                        else:
                            Read2 = 0
                    else:
                        Read2 = 0
                else:
                    Read2 = 0
            else:
                Read2 = 0
            i = i + 1
            READING2.append(Read2)
        D['READING2'] = READING2

        P = D.iloc[598,:]

        OTESTLVL = 0
        for u in levels:
            if abs(P[0]-u) <= CM:
                OTESTLVL = u

        CTESTLVL = 0
        for u in levels:
            if abs(P[3]-u) <= CM:
                CTESTLVL = u

        BREAKLVL = 0
        for u in levels:
            if P[7] == 'Bullish':
                if P[3] > (u+CM):
                    if P[0] < u:
                        BREAKLVL = u
                    elif OTESTLVL == u:
                        BREAKLVL = u
            elif P[7] == 'Bearish':
                if P[3] < (u-CM):
                    if P[0] > u:
                        BREAKLVL = u
                    elif OTESTLVL == u:
                        BREAKLVL = u
      
        F = mt5.copy_rates_from_pos(x, mt5.TIMEFRAME_M15, 0, 200)
        x1 = np.linspace(1, 200, num=200)
        y1 = F['close']
        slope, intercept, r_value, p_value, std_err = stats.linregress(x1,y1)
        wholeslope = slope

        if wholeslope > 0:
            if P[10] > 0:
                ditrend = 1
            elif P[10] < 0:
                ditrend = -1
            else:
                ditrend = 0
        elif wholeslope < 0:
            if P[10] < 0:
                ditrend = 1
            elif P[10] > 0:
                ditrend = -1
            else:
                ditrend = 0
        else:
            ditrend = 0

        if P[7] == 'Bullish':
            if P[10] > 0:
                dstrend = 1
            elif P[10] < 0:
                dstrend = -1
            else:
                dstrend = 0
        elif P[7] == 'Bearish':
            if P[10] < 0:
                dstrend = 1
            elif P[10] > 0:
                dstrend = -1
            else:
                dstrend = 0
        else:
            dstrend = 0

        ypa = ['JACK', 'GLIDE']
        grpa = ['MARIBOZU', 'TUBE', 'BOOST']
        goldpa = ['HAMMER', 'SHOOTING STAR', 'DRAGONFLY DOJI', 'GRAVESTONE DOJI', 'LING-LEG DOJI']
        silvpa = ['PLUMP DOJI', 'MALLET', 'ASTEROID', 'DIPPER', 'SKILLET']
        goldpa2 = ['BULLISH ENGULFING', 'BEARISH ENGULFING', 'TWEEZERS TOP', 'TWEEZERS BOTTOM', 'DARK CLOUD COVER', 'PIERCING PATTERN']

        color = 'white'
        if BREAKLVL != 0:
            if CTESTLVL == 0:
                if dstrend == 1:
                    color = 'pearl'
                elif dstrend == -1:
                    color = 'black'
        elif BREAKLVL == 0:
            if CTESTLVL != 0:
                if OTESTLVL == CTESTLVL:
                    if P[12] == 0:
                        color = 'gray'
                elif OTESTLVL == 0:
                    if P[12] == 0:
                        if dstrend == 1:
                            color = 'blue'
                        elif dstrend == -1:
                            color = 'purple'
                    elif P[12] != 0:
                        for ugo2 in goldpa2:
                            if P[12] == ugo2:
                                color = 'gold'
                            elif P[12] == 0:
                                if P[11] != 0:
                                    for ugo in goldpa:
                                        if P[11] == ugo:
                                            color = 'gold'
                elif OTESTLVL == 0:
                    if P[12] == 0:
                        for uy in ypa:
                            if P[11] == uy:
                                if dstrend == 1:
                                    color = 'yellow'
                                elif dstrend == -1:
                                    color = 'orange'
                        for ug in grpa:
                            if P[11] == ug:
                                if dstrend == 1:
                                    color = 'green'
                                elif dstrend == -1:
                                    color = 'red'
        
        if color == 'purple':
            coltrade = 'TRADE'
        elif color == 'gold':
            coltrade = 'TRADE'
        else:
            coltrade = 0
                
        if coltrade == 'TRADE':
            
            Hrates = mt5.copy_rates_from_pos(x, mt5.TIMEFRAME_H2, 0, 25)
            H = pd.DataFrame(Hrates)
            hlin = np.linspace(1,25,num=25)
            
            SLOPE = []
            i = 0
            for t in hlin:
                ii = i - 1
                if i == 0:
                    SLOPE.append(0)
                else:
                    slope = H.iloc[i,4] - H.iloc[ii,4]
                    SLOPE.append(slope)
                i = i + 1

            SLOPE2 = []
            i = 0
            for t in hlin:
                ii = i - 1
                if i == 0:
                    SLOPE2.append(0)
                else:
                    slope2 = SLOPE[i] - SLOPE[ii]
                    SLOPE2.append(slope2)
                i = i + 1

            CONCAVE = []
            i = 0
            for t in hlin:
                if SLOPE2[i] == 0:
                    CONCAVE.append('INFL')
                elif SLOPE2[i] > 0:
                    CONCAVE.append('UP')
                else:
                    CONCAVE.append('DOWN')
                i = i + 1

            AXCONCAVE = CONCAVE[-1]

            if P[10] > 0:
                if AXCONCAVE == 'UP':
                    contrade = 'TRADE'
                else:
                    contrade = 0
            elif P[10] < 0:
                if AXCONCAVE == 'DOWN':
                    contrade = 'TRADE'
                else:
                    contrade = 0
            else:
                contrade = 0

            if contrade == 'TRADE':
                
                if P[10] > 0:
                    position = 'LONG'
                elif P[10] < 0:
                    position = 'SHORT'
                else:
                    position = 0

                if position == 'LONG':
                    entry = CTESTLVL + CM
                elif position == 'SHORT':
                    entry = CTESTLVL - CM
                else:
                    entry = 0

                cend = int(len(levels)-1)
                if position == 'LONG':
                    clindex = levels.index(CTESTLVL)
                    tpindex = int(clindex - 1)
                    slindex = int(clindex + 1)
                    if clindex == 0:
                        stoploss = levels[slindex]
                        sldist = CTESTLVL - stoploss
                        takeprofit = CTESTLVL + sldist
                    elif clindex == cend:
                        takeprofit = levels[tpindex]
                        tpdist = takeprofit - CTESTLVL
                        stoploss = CTESTLVL - tpdist
                    else:
                        takeprofit = levels[tpindex]
                        stoploss = levels[slindex]
                elif position == 'SHORT':
                    clindex = levels.index(CTESTLVL)
                    tpindex = int(clindex + 1)
                    slindex = int(clindex - 1)
                    if clindex == 0:
                        takeprofit = levels[tpindex]
                        tpdist = CTESTLVL - takeprofit
                        stoploss = CTESTLVL + tpdist
                    elif clindex == cend:
                        stoploss = levels[slindex]
                        sldist = stoploss - CTESTLVL
                        takeprofit = CTESTLVL - sldist
                    else:
                        takeprofit = levels[tpindex]
                        stoploss = levels[slindex]
                else:
                    takeprofit = 0
                    stoploss = 0

                prp = (x,position,entry,takeprofit,stoploss,CM)
                purp.append(prp)

    purps = pd.DataFrame(purp, columns =['x','position','entry','takeprofit','stoploss','CM']) 
        
    purps['RATE'] = 0
    purps['RATE2'] = 0
    purpcnt = int(purps['x'].count())
    purplin = np.linspace(1,purpcnt,num=purpcnt)
    
    end = dt.now()
    elapsed = end - start
    elapsed = elapsed.total_seconds()
    elapsed = str(elapsed)
    timer = elapsed + '   seconds'
    print(timer)
    
    if purpcnt != 0:
        ite = 0
        while ite <= 29:
            i = 0
            for t in purplin:
                x = purps.iloc[i,0]
                position = purps.iloc[i,1]
                entry = purps.iloc[i,2]
                takeprofit = purps.iloc[i,3]
                stoploss = purps.iloc[i,4]
                CM = purps.iloc[i,5]
                k1 = purps.iloc[i,6]
                k2 = purps.iloc[i,7]
                RATE, RATE2, spy = brutus2(x,position,entry,takeprofit,stoploss,CM,4,1,k1,k2)
                purps.iloc[i,6] = RATE
                purps.iloc[i,7] = RATE2
                if spy == "NO":
                    purps.drop([i])
                i = i + 1
            ite = ite + 1
            time.sleep(10)
    
    print(purp)
    print(purps)
    end = dt.now()
    elapsed = end - start
    elapsed = elapsed.total_seconds()
    elapsed = str(elapsed)
    timer = elapsed + '   seconds'
    print(timer)
    playsound('C:/Users/joeca/Desktop/Finance/FOREX/Hercules_Prototype_Mark_II/MP3s/done.mp3')

In [8]:
#iterator = 0
#
#while iterator <= 11:
#    p = Process(target=brutus, args=())
#    p.start()
#    brutus()
#    iterator = iterator + 1
#    time.sleep(3)
#
#p.join()

In [9]:
brutus()

12.117566   seconds
[]
Empty DataFrame
Columns: [x, position, entry, takeprofit, stoploss, CM, RATE, RATE2]
Index: []
12.118526   seconds


In [10]:
t2 = 0
while True:
    Drates = mt5.copy_rates_from_pos("EURUSD", mt5.TIMEFRAME_M5, 0, 1)
    D = Drates[0]
    t = D[0]
    if t2 != 0:
        if t != t2:
            print("BRUTUS Launched")
            brutus()
    t2 = t
    time.sleep(5)

BRUTUS Launched
11.83334   seconds
[]
Empty DataFrame
Columns: [x, position, entry, takeprofit, stoploss, CM, RATE, RATE2]
Index: []
11.834337   seconds
BRUTUS Launched
11.680383   seconds
[]
Empty DataFrame
Columns: [x, position, entry, takeprofit, stoploss, CM, RATE, RATE2]
Index: []
11.680383   seconds
BRUTUS Launched
11.747098   seconds
[]
Empty DataFrame
Columns: [x, position, entry, takeprofit, stoploss, CM, RATE, RATE2]
Index: []
11.7481   seconds
BRUTUS Launched
11.898914   seconds
[]
Empty DataFrame
Columns: [x, position, entry, takeprofit, stoploss, CM, RATE, RATE2]
Index: []
11.900872   seconds


KeyboardInterrupt: 

In [ ]:
mt5.shutdown()